# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [1]:
# replace the code below with your pulse rate algorithm.
import glob
import math
import numpy as np
import scipy as sp
import scipy.signal
import matplotlib.pyplot as plt
import scipy.io



def BandpassFilter(signal, fs):
    b, a = sp.signal.butter(3,(40/60, 240/60), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)

def max_spec(signal,fs):
    spec,freq,_,_= plt.specgram(signal,Fs=fs,NFFT=fs*8,noverlap=6*fs,xextent=[0,len(signal)/60/fs])
    #freq_max=freq[np.argmax(spec,axis=0)]
    return spec,freq
    

def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    
    return data[2:]

def RunPulseRateAlgorithm(data_fl, ref_fl):
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    acc = np.sqrt(accx**2 + accy**2 + accz**2)
    
    # Compute pulse rate estimates and estimation confidence.
    fs = 125
    t= len(ppg)/fs
    ts=np.arange(0,t,1/fs)
          
    #bandpass filter 40-240bpm
    ppg_bp=BandpassFilter(ppg,fs)
    accx_bp=BandpassFilter(accx,fs)
    accy_bp=BandpassFilter(accy,fs)
    accz_bp=BandpassFilter(accz,fs)
    acc_bp=BandpassFilter(acc,fs)
       
    #--------------
    spec_ppg,freq_ppg=max_spec(ppg_bp,fs) # spec_ppg.shape=(501,148) , freq_ppg.shape=148
    spec_accx,freq_accx=max_spec(accx_bp,fs)
    spec_accy,freq_accy=max_spec(accy_bp,fs)
    spec_accz,freq_accz=max_spec(accy_bp,fs)
    spec_acc,freq_accz=max_spec(acc_bp,fs)
    
    
    freq_use = (freq_ppg> 40/60) & (freq_ppg < 240/60)
    
    spec_ppg[~freq_use]=0.0
    spec_accx[~freq_use]=0.0
    spec_accy[~freq_use]=0.0
    spec_accz[~freq_use]=0.0
    spec_acc[~freq_use]=0.0
    
    sorted_ppg=(-spec_ppg).argsort(axis=0)
    sorted_accx=(-spec_accx).argsort(axis=0)
    sorted_accy=(-spec_accy).argsort(axis=0)
    sorted_accz=(-spec_accz).argsort(axis=0)
    sorted_acc=(-spec_acc).argsort(axis=0)
    
     
    #print(spec_ppg.shape) # 501,148
    #print(freq_ok_ppg[sorted_ppg[0][1]]) # sorted array contains column wise indices max to min for column
 
    fi, confidence=[], []
    k=2 #select top 5 frequencies based on strength
    prev=freq_ppg[sorted_ppg[0][0]]
 
    for i in range(0,sorted_ppg.shape[1]):
   
        max_ppgs= freq_ppg[sorted_ppg[0:k,i]]
        #print (prev, freq_ppg[sorted_ppg[0:6,i]])
        max_accx= freq_accx[sorted_accx[0][i]]
        max_accy= freq_accy[sorted_accy[0][i]]
        max_accz= freq_accz[sorted_accz[0][i]]
        max_acc= freq_accz[sorted_acc[0][i]]
             
        idx = np.argsort(np.abs(max_ppgs - prev))
        val=prev
       
        for f in max_ppgs[idx]:
            if  (f == max_accz) or (f == max_accy) or (f==max_accx) :  
                continue
            else: 
                n=int((f-prev)/0.25)
                if abs(n)>=1 :
                      val=prev + math.floor(n/2)*0.125  # smoothening the values since heart rate values always increase gradually
                else:
                    val=f
                break
        
        fi.append(val)     
        # confidence 
        sig=spec_ppg.T[i][freq_ppg==val] # strength at the selected frequency in a particular window
        sigt=np.sum(spec_ppg.T[i]) # total strength of all frquencies in that window
        conf = sig/sigt
        confidence.append(conf[0])
        
        if i>2:
            prev=np.min(fi[-2])
    #------------------
    
    hr=[x*60 for x in fi]
    ref_data=sp.io.loadmat(ref_fl)['BPM0']
    errors=np.abs(hr-np.squeeze(ref_data))
       
    print (np.mean(errors))
       
    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return errors, np.array(confidence)